In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib import pyplot
from pylab import rcParams
import pickle as pk
import gc
import requests 
import matplotlib.gridspec as gridspec
import netCDF4 as nc

#### Import Seismic Data 

In [2]:
seismic_file = '/home/jovyan/data/hypo71.dat.txt'
df_seismic_data = pd.read_csv(seismic_file, delim_whitespace=True, dtype=object)
df_seismic_data['datetime'] = df_seismic_data['yyyymmdd'] + 'T' + \
            df_seismic_data['HHMM'].str.slice(start=0, stop=2) + ':' + \
            df_seismic_data['HHMM'].str.slice(start=2) 
df_seismic_data.index = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data['datetime'] = pd.to_datetime(df_seismic_data['datetime'].values)
df_seismic_data = df_seismic_data.loc['2017-01-1 00:00:00':'2019-06-17 00:00:00']
df_seismic_data.head()
# del df_seismic_data['yyyymmdd']
# del df_seismic_data['HHMM']
del df_seismic_data['Lon(D']
# del df_seismic_data['SSS.SS']
# del df_seismic_data['Depth']
del df_seismic_data['M)']
del df_seismic_data['M).1']
del df_seismic_data['NWR']
del df_seismic_data['GAP']
del df_seismic_data['DMIN']
del df_seismic_data['ERH']
del df_seismic_data['ERZ']
del df_seismic_data['ID']
del df_seismic_data['Lat(D']
del df_seismic_data['PMom']
del df_seismic_data['SMom']
df_seismic_data['Depth'] = df_seismic_data['Depth'].astype('float64').values
df_seismic_data['MW'] = df_seismic_data['MW'].astype('float64').values
df_seismic_data['RMS'] = df_seismic_data['RMS'].astype('float64').values

In [ ]:
df_seismic_data.datetime.astype(np.int64).values/1e64
df_seismic_data['date'] =pd.DatetimeIndex(df_seismic_data.datetime).date
df_seismic_data.tail()

In [ ]:
df_seismic_data.plot(x='datetime', y= 'Depth')

In [ ]:
df_seismic_data.plot(x='datetime', y= 'MW')

In [ ]:
df_eqMean=df_seismic_data.groupby('date').mean()
df_eqCount= df_seismic_data.groupby('date').count()
#del df_eqMean['datetime']
#df_eqMean.columns.name = df_eqMean.index.name
#df_eqMean.index.name = None
df_eqMean.tail()

In [ ]:
df_eqCount['count'] = df_eqCount.datetime.astype('float64').values
df_eqCount.head()

In [ ]:
df_eqMean['count'] = df_eqCount['count'].values
df_eqMean.head()

In [ ]:
idx = pd.date_range('2017-01-1 00:00:00', '2019-06-17 00:00:00')

df_eqMean = df_eqMean

df_eqMean.index = pd.DatetimeIndex(df_eqMean.index)

df_eqMean = df_eqMean.reindex(idx, fill_value=0)
df_eqMean.tail()

In [ ]:
import hvplot.pandas
a= df_eqMean.hvplot.scatter(y= 'count')
b= df_eqMean.hvplot.scatter(y= 'Depth')
a+b

In [ ]:
df_eqMean.hvplot.scatter(y= ['count', 'Depth','MW','RMS']
                        , subplots=True, shared_axes=False).cols(1)

#### Create time and count vectors earthquake frequency vectors for plotting 

In [ ]:
count = df_eqMean['count'].tolist()
time_int = []
time = list(pd.to_datetime(df_eqMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
# smooth data using rolling window that chops off 95th percentile 
count_av = list(movingaverage(df_eqMean['count'],21))

#### Create time and count vectors earthquake frequency vectors for plotting 

In [ ]:
count = df_eqMean['count'].tolist()
time_int = []
time = list(pd.to_datetime(df_eqMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
# smooth data using rolling window that chops off 95th percentile 
count_av = list(movingaverage(df_eqMean['count'],21))

#### Create time and MW vectors for plotting 

In [ ]:
MW = df_eqMean['MW'].tolist()
time_int = []
time = list(pd.to_datetime(df_eqMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
# smooth data using rolling window that chops off 95th percentile 
MW_av = list(movingaverage(df_eqMean['MW'],21))

#### Create time and depth vectors for plotting

In [ ]:
depth = df_eqMean['Depth'].tolist()
time_int = []
time = list(pd.to_datetime(df_eqMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [ ]:
# smooth data using rolling window that chops off 95th percentile 
depth_av = list(movingaverage(df_eqMean['Depth'],21))

#### Read tide data

In [ ]:
tide_file = '/home/jovyan/data/tide/pred_F.txt'
df_tides = pd.read_csv(tide_file, delim_whitespace=True, dtype = object)
df_tides['datetime']=df_tides['year'] + '-' + df_tides['month'] + '-' + df_tides['day'] + \
            'T' + df_tides['hour'] + ':' + df_tides['minute'] + ':' + df_tides['second']
df_tides.index=pd.to_datetime(df_tides['datetime'].values)
del df_tides['year']
del df_tides['month']
del df_tides['day']
del df_tides['hour']
del df_tides['minute']
del df_tides['second']
del df_tides['datetime']
df_tides['height'] = df_tides['height'].astype(float)
df_tides= df_tides.sort_index()

In [ ]:
df_tides.tail()

#### Create time and height vectors for plotting 

In [ ]:
height = df_tides['height'].tolist()
time_int = []
time = list(pd.to_datetime(df_tides.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
epoch= [i.timestamp() for i in df_tides.index.to_pydatetime()]
df_tides['epoch'] = epoch
df_tides= df_tides.sort_index()
df_tides.head()

#### Use Groupby to create one day mean measurements

In [ ]:
tides = df_tides['height'].tolist()
time_int = []
time = list(pd.to_datetime(df_tides.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
# Plot one day 
plt.close()
fig, ax = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax.plot(time_int, height)
ax.yaxis.grid(True)
ax.xaxis.grid(True)
ax.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 17, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
ax.xaxis.set_minor_locator(years)
ax.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax.xaxis.get_majorticklabels(), rotation=90)
ax.set_title('Tide Height', fontsize=36, fontweight = 'bold')
ax.set_ylabel('Tide height (m)', fontsize=20, fontweight = 'bold', rotation=90)

#plt.gca().invert_yaxis()
plt.savefig('/home/jovyan/botpt/figures/tides.png')
plt.show()

In [ ]:
df_tides['date']=pd.DatetimeIndex(df_tides.index).date
#df_botptMerge.head()

In [ ]:
df_tides = df_tides.loc['2017-01-1 00:00:00':'2019-06-17 00:00:00']
df_tides.tail()

#### Use Groupby to create one day mean measurements

In [ ]:
df_tidesMean=df_tides.groupby('date').mean()
df_tidesMean.head()

In [ ]:
# time = list(df_botptMerge.index.values)
#height = x.tolist()
height = df_tidesMean['height'].tolist()
time_int = []
time = list(pd.to_datetime(df_tidesMean.index.values))
for i in time:
    i = np.datetime64(i).astype(datetime.datetime)
    time_int.append(dates.date2num(i))

In [ ]:
# df_tidesMean['date'] = date
# df['Address'] = address 

In [ ]:
# Plot one day 
plt.close()
fig, ax1 = plt.subplots()
fig.set_size_inches(28, 6)
hb1 = ax1.plot(time_int, height)
ax1.yaxis.grid(True)
ax1.xaxis.grid(True)
ax1.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 11, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax1.xaxis.set_major_locator(months)
ax1.xaxis.set_major_formatter(monthsFmt)
ax1.xaxis.set_minor_locator(years)
ax1.xaxis.set_minor_formatter(yearsFmt)
plt.tight_layout()
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
ax1.set_title('Mean Tide Height', fontsize=36, fontweight = 'bold')
ax1.set_ylabel('Tide height (m)', fontsize=20, fontweight = 'bold', rotation=90)
# plt.savefig('/home/jovyan/botpt/figures/meantides.png')
#plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.close()
fig3, (ax1,ax2,ax3,ax4) = plt.subplots(4,1)
fig3.set_size_inches(30, 14)

hb1 = ax1.plot(time_int, height, linewidth=4)
ax1.yaxis.grid(True)
ax1.xaxis.grid(True)
ax1.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 17, 0, 0))
#years = dates.YearLocator()
months = dates.MonthLocator()
#yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax1.xaxis.set_major_locator(months)
ax1.xaxis.set_major_formatter(monthsFmt)
#ax1.xaxis.set_minor_locator(years)
#ax1.xaxis.set_minor_formatter(yearsFmt)
ax1.set_title('Mean Tides Height', fontsize=24, fontweight = 'bold')
ax1.set_ylabel('Height (m)', fontsize=20, fontweight = 'bold', rotation=90)


hb1 = ax2.plot(time, count, linewidth=4)
hb2 = ax2.plot(time, count_av, linewidth=3)
ax2.yaxis.grid(True)
ax2.xaxis.grid(True)
ax2.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 17, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax2.xaxis.set_major_locator(months)
ax2.xaxis.set_major_formatter(monthsFmt)
ax2.xaxis.set_minor_locator(years)
ax2.xaxis.set_minor_formatter(yearsFmt)
ax2.set_title('Daily Seismicity', fontsize=24, fontweight = 'bold')
ax2.set_ylabel('EQ/day', fontsize=20, fontweight = 'bold', rotation=90)

hb1 = ax3.plot(time, MW)
hb2 = ax3.plot(time, MW_av, linewidth=5)
ax3.yaxis.grid(True)
ax3.xaxis.grid(True)
ax3.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 17, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax3.xaxis.set_major_locator(months)
ax3.xaxis.set_major_formatter(monthsFmt)
ax3.xaxis.set_minor_locator(years)
ax3.xaxis.set_minor_formatter(yearsFmt)
ax3.set_title('Earthquake Magnitude', fontsize=24, fontweight = 'bold')
ax3.set_ylabel('MW', fontsize=20, fontweight = 'bold', rotation=90)

hb1 = ax4.plot(time, depth)
hb2 = ax4.plot(time, depth_av, linewidth=5)
ax4.yaxis.grid(True)
ax4.xaxis.grid(True)
ax4.set_xlim(datetime.datetime(2017, 1, 1, 0, 0),datetime.datetime(2019, 6, 17, 0, 0))
years = dates.YearLocator()
months = dates.MonthLocator()
yearsFmt = dates.DateFormatter('\n\n\n%Y')
monthsFmt = dates.DateFormatter('%b')
ax4.xaxis.set_major_locator(months)
ax4.xaxis.set_major_formatter(monthsFmt)
ax4.xaxis.set_minor_locator(years)
ax4.xaxis.set_minor_formatter(yearsFmt)
ax4.set_title('Earthquake Depth', fontsize=24, fontweight = 'bold')
ax4.set_ylabel('Depth (km)', fontsize=20, fontweight = 'bold', rotation=90)

#plt.tight_layout()
plt.setp(ax1.set_xticklabels([]))
plt.setp(ax2.set_xticklabels([]))
plt.setp(ax3.set_xticklabels([]))

#plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, fontsize=24)
#plt.setp(ax1.xaxis.get_minorticklabels(), rotation=0, fontsize=24)
plt.setp(ax1.yaxis.get_majorticklabels(), rotation=0, fontsize=20, fontweight = 'bold')
plt.setp(ax2.yaxis.get_majorticklabels(), rotation=0, fontsize=20, fontweight = 'bold')
plt.setp(ax3.yaxis.get_majorticklabels(), rotation=0, fontsize=20, fontweight = 'bold')
plt.setp(ax4.yaxis.get_majorticklabels(), rotation=0, fontsize=20, fontweight = 'bold')

plt.setp(ax2.yaxis.get_minorticklabels(), rotation=0, fontsize=20, fontweight = 'bold')
plt.setp(ax3.yaxis.get_minorticklabels(), rotation=0, fontsize=20, fontweight = 'bold')
plt.setp(ax4.xaxis.get_minorticklabels(), rotation=0, fontsize=20, fontweight = 'bold')

plt.suptitle('Comparison Between Tides and Earthquakes',
            fontsize=32, color= 'blue', fontweight = 'bold')
plt.subplots_adjust(top=0.90, hspace=0.25)
plt.savefig('/home/jovyan/botpt/figures/tidesvseq.png')